In [1]:
import numpy as np

In [2]:
np.random.seed(0)

In [3]:
data = "hello world"
chars = list(set(data))
vocab_size = len(chars)

chars_to_indices = {char: i for i, char in enumerate(chars)}
indices_to_chars = {i: char for i, char in enumerate(chars)}

chars_to_indices, indices_to_chars

({'o': 0, 'l': 1, 'w': 2, 'd': 3, 'h': 4, ' ': 5, 'e': 6, 'r': 7},
 {0: 'o', 1: 'l', 2: 'w', 3: 'd', 4: 'h', 5: ' ', 6: 'e', 7: 'r'})

In [4]:
# RNN configs
embedding_size = vocab_size
hidden_size = 64
output_size = vocab_size

In [5]:
# Matrices
Wxh = np.random.randn(embedding_size, hidden_size)
Whh = np.random.randn(hidden_size, hidden_size)
bh = np.random.randn(1, hidden_size)

Why = np.random.randn(hidden_size, output_size)
by = np.random.randn(1, output_size)

In [6]:
def softmax(z):
    e_z = np.exp(z)
    return e_z / e_z.sum(axis=1)

def rnn(x_t, h_prev):
    assert x_t.shape == (1, embedding_size)
    assert h_prev.shape == (1, hidden_size)

    h_next = np.tanh(x_t @ Wxh + h_prev @ Whh + bh)
    probs = softmax(h_next @ Why + by)
    
    return probs, h_next

def sample(char, n):
    # sample rnn n times starting with the first char
    x = np.zeros((1, vocab_size))
    idx = chars_to_indices[char]
    x[:, idx] = 1 # one hot encoding
    h =  np.zeros((1, hidden_size)) # hidden state
    idxes = []
    
    for i in range(n):
        probs, h = rnn(x, h) # xt, hprev -> rnn -> probs, hnext
        idx = np.random.choice(vocab_size, p=probs.ravel()) # .ravel returns a 1d array
        
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        idxes.append(idx)

    chars = "".join([indices_to_chars[i] for i in idxes])
    return chars

In [7]:
x_t = np.random.randn(1, embedding_size)
h_prev = np.random.randn(1, hidden_size)

probs, h_next = rnn(x_t, h_prev)
probs.shape, h_next.shape

((1, 8), (1, 64))

In [8]:
inp = [chars_to_indices[c] for c in data[:-1]]
target = [chars_to_indices[c] for c in data[1:]]

inp, target

([4, 6, 1, 1, 0, 5, 2, 0, 7, 1], [6, 1, 1, 0, 5, 2, 0, 7, 1, 3])

In [9]:
sample("h", 4)

'dlle'

In [15]:
lr = 1e-2
for e in range(100):
    # compute loss
    loss = 0
    xs, ps, hs = {}, {}, {}
    h = np.zeros((1, hidden_size))
    for t in range(len(inp)):
        idx = inp[t]
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        probs, h = rnn(x, h)
        xs[t] = x
        ps[t] = probs # save probs (we'll use this in backprop)
        hs[t] = h

        # cross entropy
        pred = probs[0, target[t]]
        loss += -np.log(pred)

    # backprop, calculate gradients
    dL_dWxh = np.zeros_like(Wxh)
    dL_dWhh = np.zeros_like(Whh)
    dL_dbh = np.zeros_like(bh)
    dL_Why = np.zeros_like(Why)
    dL_dby = np.zeros_like(by)
    dF_dh = np.zeros((1, hidden_size))

    for t in reversed(range(len(inp))):
        dL_dz2 = np.copy(ps[t])
        target_idx = target[t]
        dL_dz2[:, target_idx] -= 1

        # 2nd layer
        dL_Why += dL_dz2 * hs[t].T
        dL_dby += dL_dz2

        # 1st layer
        dh_dz1 = 1 - hs[t] ** 2
        dL_dh = (dL_dz2 @ Why.T) + dF_dh 

        dL_dWxh += dL_dh * dh_dz1 * xs[t].T
        dL_dWhh += dL_dh * dh_dz1 * hs[t].T
        dL_dbh += dL_dh * dh_dz1

        dF_dh = dL_dh * (dh_dz1 @ Whh.T)

    # clip gradients
    for gradient in [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]:
        np.clip(gradient, -5, 5, out=gradient)

    # gradient descent
    for weights, gradient in zip([Wxh, Whh, Why, bh, by], [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]):
        weights -= lr * gradient

    # break
    
    print(f"[Loss]: {loss}")

[Loss]: 8.697269623130946
[Loss]: 8.659623627242965
[Loss]: 4.396628578245165
[Loss]: 3.332594101688114
[Loss]: 6.279528819870177
[Loss]: 8.108745645767254
[Loss]: 3.8869127432265125
[Loss]: 9.153459124345744
[Loss]: 10.00123781341093
[Loss]: 4.608065546306275
[Loss]: 2.637066750128036
[Loss]: 4.236364009469662
[Loss]: 10.59848304170207
[Loss]: 8.447979742873839
[Loss]: 3.464235576686472
[Loss]: 5.4149597235891465
[Loss]: 7.451926769267697
[Loss]: 12.851299357839268
[Loss]: 3.681721100962129
[Loss]: 6.021301945482022
[Loss]: 3.5084735200098365
[Loss]: 7.485607584536385
[Loss]: 5.343663294856562
[Loss]: 2.3447480614950598
[Loss]: 1.5885425983602253
[Loss]: 2.5235092740111833
[Loss]: 2.2157086550252654
[Loss]: 8.84539867419696
[Loss]: 3.662516673835035
[Loss]: 11.918414188352912
[Loss]: 3.3743649806053413
[Loss]: 0.6746442993443703
[Loss]: 0.6063288595528465
[Loss]: 0.652839142851938
[Loss]: 0.9428636001353383
[Loss]: 1.3551515852625169
[Loss]: 4.085518799292492
[Loss]: 0.562455422690127

In [16]:
sample("h", 10)

'ello world'